# Agentic Framework in Llama Index

## Tool Use

The FunctionTool module can be used here to convert any function into a tool that the agent could call if it feels the necessity



In [32]:
import requests
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import ReActAgent

from llama_index.llms.openai import OpenAI
import nest_asyncio
from llama_index.agent.introspective import IntrospectiveAgentWorker
from llama_index.agent.introspective import (
    ToolInteractiveReflectionAgentWorker,
)
import os

import dotenv


from llama_index.agent.openai import OpenAIAgentWorker
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate

nest_asyncio.apply()

dotenv.load_dotenv()

True

In [33]:
# Load API Keys
API_KEY = os.getenv('OPENWEATHER_API_KEY')
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [34]:
print(API_KEY)

41640c91aff77e824e7f7a11f984c49a


### 1. To access external API data 

In [11]:
def get_weather(city_name, units='imperial'):
    """
    Function that taken in a city name and units and returns the temperature and description of weather

    Args:
    city_name: The name of the city (ex: Los Angeles)
    units: The measurement units for tempreature (Options: imperial - fahreinheit, metric - celsius), Default: fahrenheit (imperial)

    Return:
    city: Name of the City
    temperature: the temperature in the provided measurement units
    description: the short description of the weather
    """
    base_url = "http://api.openweathermap.org/data/2.5/weather?"
    complete_url = base_url + "appid=" + API_KEY + "&q=" + city_name + "&units=" + units
    
    response = requests.get(complete_url)
    
    if response.status_code == 200:
        weather_data = response.json()
        return {
            'city': city_name,
            'temperature': weather_data['main']['temp'],
            'description': weather_data['weather'][0]['description']
        }
    else:
        return {'error': 'Failed to retrieve weather data', 'status_code': response.status_code}

In [14]:
# Declare tool
weather_fetch_tool = FunctionTool.from_defaults(fn=get_weather)

#### Instance of simple function call using the OpenAI LLM

In [16]:
# Use query engine with the tool

llm = OpenAI(model="gpt-3.5-turbo", api_key=OPENAI_API_KEY)
response = llm.predict_and_call(
    [weather_fetch_tool], 
    "How is the weather in San Jose right now?", 
    verbose=True
)
print(str(response))

=== Calling Function ===
Calling function: get_weather with args: {"city_name": "San Jose"}
=== Function Output ===
{'city': 'San Jose', 'temperature': 76.59, 'description': 'scattered clouds'}
{'city': 'San Jose', 'temperature': 76.59, 'description': 'scattered clouds'}


#### Using the agent

### Combination of Runner and Worker

Runner - Manages/Orchestrates the different tasks once the query is received, oversees the outputs of the worker, returns final response when ready

Worker - Does the actual execution of the individual tasks using the available tools

![components](/Users/chandrasekaransidha/personal/sandbox/img/agent_components.png)

In [17]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    [weather_fetch_tool], 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)


In [18]:
response = agent.query("Who is Connor McGregor?")

Added user message to memory: Who is Connor McGregor?
=== LLM Response ===
Connor McGregor is a professional mixed martial artist and former UFC (Ultimate Fighting Championship) featherweight and lightweight champion. He is known for his charismatic personality, trash-talking, and fighting skills inside the octagon. McGregor is one of the most popular and successful fighters in the history of MMA (Mixed Martial Arts).


In [20]:
response = agent.query("Is it a good day to go running outside, I'm in Hayward?")

Added user message to memory: Is it a good day to go running outside, I'm in Hayward?
=== Calling Function ===
Calling function: get_weather with args: {"city_name": "Hayward"}
=== Function Output ===
{'city': 'Hayward', 'temperature': 68.27, 'description': 'clear sky'}
=== LLM Response ===
The weather in Hayward is currently 68.27°F with a clear sky. It seems like a good day to go running outside!


In [22]:
# Using the ReAct Agent

agent = ReActAgent.from_tools([weather_fetch_tool], llm=llm, verbose=True)


response = agent.chat("Is it a good day to go running outside, I'm in Hayward?")

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: get_weather
Action Input: {'city_name': 'Hayward', 'units': 'imperial'}
Observation: {'city': 'Hayward', 'temperature': 68.27, 'description': 'clear sky'}
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The current temperature in Hayward is 68.27°F with clear skies. It seems like a good day to go running outside!


### 2. Auto Retrieval: To perform metadata filtering while retrieving data (RAG) 